# メタプログラミング
自分自身をデータをデータとして扱うことのできるコンピュータプログラミング

実行中に自身へのイントロスペクションを行ったり，コード生成や修正が可能

- 関数，クラス，型などの基本要素をその場で作成したり変更する  ex)デコレータ

- クラスのインスタンス作成プロセスに干渉する特別なクラスメソッド

## 3.4.1 デコレータ

In [2]:
def some_decorator(function):
    return function
    
def decorated_function():
    pass
decorated_function = some_decorator(decorated_function)

## 3.4.2 クラスデコレータ
関数ではなくクラスを返すことを期待されているデコレ―タ
- インスタンスだけでなくクラスも実行時に変更できる
- 関数もディスクリプタであるため，ディスクリプタプロトコルの一部としてインスタンスに対する属性のルックアップが行われるときに実際のインスタンスへのバインドが行われるため，実行時にクラスに追加することが可能
- 適切に引数が渡されたsuperは，クラス定義のスコープ外でも利用できる
- クラス定義に対してデコレータを呼び出すことが出来る


In [8]:
def short_repr(cls):
    cls.__repr__= lambda self: super(cls,self).__repr__()[:15]
    return cls
@ short_repr
class ClassWithRelativelyLoongName:
    pass
ClassWithRelativelyLoongName()

<__main__.Class

In [17]:
# 動的に作成されるクラスになるので__name__と__doc__属性に影響を与える
# mixinクラスデザインの代替になる(少ない労力で多重継承を避けれる)

def parametraized_short_repr(max_width=8):
    """文字列表現を短縮するパラメータ付きデコレータ"""
    def parametarized(cls):
        """実際のデコレータとして使用される内部ラッパー関数"""
        class ShortlyRepresented(cls):
            """デコレートされた動作を提供するサブクラス"""
            def __repr__(self):
                return super().__repr__()[:max_width]
        return ShortlyRepresented
    return parametarized

@parametraized_short_repr(15)
class ClassWithLittleBitLongerName:
    pass


In [29]:
ClassWithLittleBitLongerName().__class__

__main__.parametraized_short_repr.<locals>.parametarized.<locals>.ShortlyRepresented

In [28]:
ClassWithLittleBitLongerName().__doc__

'デコレートされた動作を提供するサブクラス'

## 3.4.3 __new__ メソッドによるインスタンス作成プロセスのオーバーライド
newはクラスのインスタンスを生成する責務を持った静的メソッド．initよりも前に呼び出される

In [ ]:
# 通常は，オーバーライドしたnewメソッドの中で，
# super.new()のように親クラスを引数付きで呼び出し，
# 帰ってきたインスタンスをreturnする前に変更する

class InstanceCountingClass:
    instance_created = 0
    def __new__(cls,*args,**kwargs):
        print("__new__が呼ばれました：",cls,args,**kwargs)
        instance= super().__new__(cls)
        instance.number = cls.instance_created
        cls.instance_created += 1
        
        return instance
    
    def __init__(self,attribute):
        print("__init__()が呼ばれました．：",self,attribute)
        self.attribute = attribute
        

In [ ]:
instance1 = InstanceCountingClass("abc")

In [ ]:
instance2 = InstanceCountingClass("xyz")

In [ ]:
instance1.number,instance1.instance_created

In [ ]:
instance2.number,instance2.instance_created

In [ ]:
# newメソッドは通常，対象クラスのインスタンスを返すべきだが，他のクラスも返せる
# その場合initメソッドの呼び出しはスキップされる
# 不変クラスのインスタンスを作成するときに有用

class NoneZero(int):
    def __new__(cls,value):
        return super().__new__(cls,value) if value !=0 else None
    def __init__(self,skipped_value):
        # この実装部分はスキップされる可能性がある
        print("__init__が呼ばれました")
        super().__init__()
type(NoneZero(0))
        

In [ ]:
NoneZero(-3.123)

new はinit で不十分なときに使用する

不変オブジェクトのインスタンスをinitメソッド内ではすでに作成されており，変更できないため



## 3.4.4 メタクラス
メタクラスとは他の型（クラス）を定義する型（クラス） ⇒ オブジェクトのインスタンスを定義するくらすもまたオブジェクト

instance <is instance of> class <is instance of> type
instance <is inctance of> class <is instance of> metaclass <is SUBCLASS of> type 
    
組み込みのtype()クラスを呼び出すとクラス構文を使ってクラスを作成するのと同等の事が出来る

In [ ]:
def method(self):
    return 1
klass = type("MyClass",(object,),{"method":method})

instance = klass()
instance.method()

In [ ]:
# クラス構文を使うと暗黙的にメタクラスとしてTypeを指定したことになる
class ClassWithAMetaClass(metaclass=type):
    pass
# metaclass引数はTypeクラスと同じ引数を受け取り別のクラスオブジェクトを返せるCallable

In [ ]:
# メタクラス共通のテンプレート

class Metaclass(type):
    def __new__(mcs,name,bases,namespace):
        return super().__name__(mcs,name,bases,namespace)
    @classmethod
    def __prepare__(mcs,name,bases,**kwargs):
        return super().__prepare__(mcs,name,bases,**kwargs)
    def __init__(mcs,name,bases,namespace,**kwargs):
        super().__init__(name,bases,namespace)
    def __call__(cls,+args.**kwargs):
        return super().__call__(*arg,**kwargs)

In [3]:
# メタクラス共通のテンプレート

class Metaclass(type):
    def __new__(mcs,name,bases,namespace):
        return super().__name__(mcs,name,bases,namespace)
    @classmethod
    def __prepare__(mcs,name,bases,**kwargs):
        return super().__prepare__(mcs,name,bases,**kwargs)
    def __init__(mcs,name,bases,namespace,**kwargs):
        super().__init__(name,bases,namespace)
    def __call__(cls,+args.**kwargs):
        return super().__call__(*arg,**kwargs)

In [4]:
instance1 = InstanceCountingClass("abc")

__new__が呼ばれました： <class '__main__.InstanceCountingClass'> ('abc',)
__init__()が呼ばれました．： <__main__.InstanceCountingClass object at 0x000002BD65297B70> abc


In [5]:
instance2 = InstanceCountingClass("xyz")

__new__が呼ばれました： <class '__main__.InstanceCountingClass'> ('xyz',)
__init__()が呼ばれました．： <__main__.InstanceCountingClass object at 0x000002BD652972B0> xyz


In [6]:
instance1.number,instance1.instance_created

(0, 2)

In [7]:
instance2.number,instance2.instance_created

(1, 2)

In [12]:
# newメソッドは通常，対象クラスのインスタンスを返すべきだが，他のクラスも返せる
# その場合initメソッドの呼び出しはスキップされる
# 不変クラスのインスタンスを作成するときに有用

class NoneZero(int):
    def __new__(cls,value):
        return super().__new__(cls,value) if value !=0 else None
    def __init__(self,skipped_value):
        # この実装部分はスキップされる可能性がある
        print("__init__が呼ばれました")
        super().__init__()
type(NoneZero(0))
        

NoneType

In [13]:
NoneZero(-3.123)

__init__が呼ばれました


-3

new はinit で不十分なときに使用する

不変オブジェクトのインスタンスをinitメソッド内ではすでに作成されており，変更できないため



## 3.4.4 メタクラス
メタクラスとは他の型（クラス）を定義する型（クラス） ⇒ オブジェクトのインスタンスを定義するくらすもまたオブジェクト

instance <is instance of> class <is instance of> type
instance <is inctance of> class <is instance of> metaclass <is SUBCLASS of> type 
    
組み込みのtype()クラスを呼び出すとクラス構文を使ってクラスを作成するのと同等の事が出来る

In [14]:
def method(self):
    return 1
klass = type("MyClass",(object,),{"method":method})

instance = klass()
instance.method()

1

In [15]:
# クラス構文を使うと暗黙的にメタクラスとしてTypeを指定したことになる
class ClassWithAMetaClass(metaclass=type):
    pass
# metaclass引数はTypeクラスと同じ引数を受け取り別のクラスオブジェクトを返せるCallable

In [16]:
# メタクラス共通のテンプレート

class Metaclass(type):
    def __new__(mcs,name,bases,namespace):
        return super().__name__(mcs,name,bases,namespace)
    @classmethod
    def __prepare__(mcs,name,bases,**kwargs):
        return super().__prepare__(mcs,name,bases,**kwargs)
    def __init__(mcs,name,bases,namespace,**kwargs):
        super().__init__(name,bases,namespace)
    def __call__(cls,+args.**kwargs):
        return super().__call__(*arg,**kwargs)

SyntaxError: invalid syntax (<ipython-input-16-903c7426d7b6>, line 11)